In [ ]:
import numpy as np
from sklearn import tree
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd
import graphviz
import matplotlib.pyplot as plt

In [ ]:
# Parameters
random_state = 42
min_samples_leaf = 5

## 1) Get an artificial inflation of some class in the training set by a given factor: 10 (weigh more the classes virginica e versicolor which are more difficult to discriminate). Learn the tree in these conditions.

In [ ]:
# data, target = load_iris(return_X_y=True)
iris = load_iris()
inflation_factor = 10

# Compute the repetitions of each elem of the dataset
repetitions = np.clip(iris.target * inflation_factor, a_min=1, a_max=inflation_factor)

# Apply inflation to dataset
inflated_data = np.repeat(iris.data, repetitions, axis=0)
inflated_target = np.repeat(iris.target, repetitions, axis=0)

# Get train & test datasets
X_train, X_test, y_train, y_test_inf = train_test_split(inflated_data, inflated_target, test_size=0.2, random_state=random_state)

# Learning of the tree
clf_inflated = tree.DecisionTreeClassifier(criterion="entropy", random_state=random_state, min_samples_leaf=5)
clf_inflated = clf_inflated.fit(X_train, y_train)

# Get predictions
y_pred_inf = clf_inflated.predict(X_test)

# Show pairs
pd.DataFrame(np.column_stack((y_pred_inf, y_test_inf)).T, index=['prediction', 'reference'])

### Evaluation

In [ ]:
accuracy = accuracy_score(y_test_inf, y_pred_inf)
report = classification_report(y_test_inf, y_pred_inf, labels=[0, 1, 2])

print(f'Accuracy: {accuracy * 100}%')
print(report)

### Graph

In [ ]:
dot_data = tree.export_graphviz(clf_inflated, out_file=None, feature_names=iris.feature_names, class_names=iris.target_names, filled=True, rounded=True, special_characters=True)  
graph = graphviz.Source(dot_data)  
graph

## 2) Modify the weight of some classes (set to 10 the weights for misclassification between virginica into versicolor and vice versa) and learn the tree in these conditions. You should obtain similar results as for step 1.

In [ ]:
X,y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test_w = train_test_split(X, y, test_size=0.2, random_state=7)
clf_weighted = tree.DecisionTreeClassifier(criterion="entropy", random_state=21, min_samples_leaf=min_samples_leaf, class_weight={0:1,1:10,2:10})

# Learning of the tree applying weights
clf_weighted.fit(X_train, y_train)

# Get predictions
y_pred_w = clf_weighted.predict(X_test)

# Show pairs
pd.DataFrame(np.column_stack((y_pred_w, y_test)).T, index=['prediction', 'reference'])

### Evaluation

In [ ]:
accuracy = accuracy_score(y_test_w, y_pred_w)
report = classification_report(y_test_w, y_pred_w, labels=[0, 1, 2])

print(f'Accuracy: {accuracy * 100}%')
print(report)

### Graph

In [ ]:
dot_data = tree.export_graphviz(clf_weighted, out_file=None, feature_names=iris.feature_names, class_names=iris.target_names, filled=True, rounded=True, special_characters=True)  
graph = graphviz.Source(dot_data)  
graph

## 3) Learn trees but avoid overfitting (by improving the error on the test set) tuning the parameters on: the minimum number of samples per leaf, max depth of the tree, min_impurity_decrease parameters, max leaf nodes, etc.

In [ ]:
X,y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

hyperparameters = list()

for min_samples_l in range(2, 15):
    for max_leaf_n in range(2, 15):
        for min_samples_s in range(2, 30):
            for impurity in np.linspace(0, 1, 21):
                clf_improved = tree.DecisionTreeClassifier(criterion="entropy", random_state=random_state, min_samples_leaf=min_samples_l, max_leaf_nodes=max_leaf_n, min_samples_split=min_samples_s, min_impurity_decrease=impurity)
                clf_improved = clf_improved.fit(X_train, y_train)
                y_pred_imp = clf_improved.predict(X_test)

                accuracy = accuracy_score(y_test, y_pred_imp)
                f1 = f1_score(y_test, y_pred_imp, average='macro')
                if accuracy + f1 == 2:
                    hyperparameters.append([min_samples_l, max_leaf_n, min_samples_s, impurity])

In [ ]:

clf_improved = tree.DecisionTreeClassifier(criterion="entropy", random_state=random_state, min_samples_leaf=hyperparameters[0][0], max_leaf_nodes=hyperparameters[0][1], min_samples_split=hyperparameters[0][2], min_impurity_decrease=hyperparameters[0][3])
clf_improved = clf_improved.fit(X_train, y_train)
y_pred_imp = clf_improved.predict(X_test)

In [ ]:
mat = np.array(hyperparameters)
df = pd.DataFrame(mat, columns=["min_samples_leaf", "max_leaf_nodes", "min_samples_split", "min_impurity_decrease"])
df.sort_values(["min_samples_leaf", "max_leaf_nodes", "min_samples_split", "min_impurity_decrease"], inplace=True)
df.to_html("optimized_hyperparameters.html")

Reading the table created with the optimized hyperparameters we see that:
* <b>min_samples_leaf</b>: from 2 to 8
* <b>max_leaf_nodes</b>: from 4 to 14
* <b>min_samples_split</b>: from 2 to 29
* <b>min_impurity_decrease</b>: 0 to 0.05

## 4) Build the confusion matrix of the created tree models on the test set and show them. 

In [ ]:
inflated_confusion_matrix = confusion_matrix(y_true=y_test_inf, y_pred=y_pred_inf)
weighted_confusion_matrix = confusion_matrix(y_true=y_test, y_pred=y_pred_w)
improved_confusion_matrix = confusion_matrix(y_true=y_test, y_pred=y_pred_imp)

print(pd.DataFrame(inflated_confusion_matrix, index=iris.target_names, columns=iris.target_names))
print(pd.DataFrame(weighted_confusion_matrix, index=iris.target_names, columns=iris.target_names))
print(pd.DataFrame(improved_confusion_matrix, index=iris.target_names, columns=iris.target_names))

## 5) Build the ROC curves (or coverage curves in coverage space) and plot them for each tree model you have created: for each model you have to build three curves, one for each class, considered in turn as the positive class.